In [1]:
import os

os.chdir('/mnt/jw01-aruk-home01/projects/ra_challenge/RA_challenge/michael_dev/RA2_alpine_lads/ra_joint_predictions')

In [2]:
from utils.config import Config

config = Config()

In [3]:
from dataset.joint_val_dataset import hands_joints_val_dataset, feet_joint_val_dataset, hands_wrists_val_dataset
from dataset.joints.joint_extractor import default_joint_extractor
from dataset.joints.joint_extractor_factory import get_joint_extractor

hands_ds = hands_joints_val_dataset(config, joint_extractor = get_joint_extractor('H', False), split_type = 'balanced')
feet_ds = feet_joint_val_dataset(config, joint_extractor = get_joint_extractor('F', True), split_type = 'balanced')
wrists_ds = hands_wrists_val_dataset(config)

tf_hands_data = hands_ds.create_hands_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
tf_hands_e_data = hands_ds.create_hands_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)
tf_feet_j_data = feet_ds.create_feet_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
tf_feet_e_data = feet_ds.create_feet_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)
wrist_j_data = wrists_ds.create_wrists_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = False)
wrist_e_data = wrists_ds.create_wrists_joints_dataset_with_validation(config.train_location + '/training.csv', erosion_flag = True)

2020-05-10 10:48:57,920;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:48:58,072;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:48:58,303;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:48:58,453;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:48:58,606;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:02,258;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:02,409;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:02,560;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:02,714;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:02,992;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:03,993;WARNING - Missing permissions to create directory for caching!
2020-05-10 10:49:04,144;WARNING - Missing p

In [29]:
import numpy as np

from tensorflow import keras
from model.utils.metrics import mae_metric, rmse_metric, class_filter_rmse_metric, softmax_mae_metric, softmax_rmse_metric, class_filter_softmax_rmse_metric

def eval_model(label, dataset, model_path, max_output, steps, filter_model_paths = None, filter_cutoff = 0.5):
    model = keras.models.load_model(model_path, compile = False)
    
    if filter_model_paths is not None:
        filter_models = [keras.models.load_model(filter_model_path, compile = False) for filter_model_path in filter_model_paths]
    else:
        filter_models = None
    
    truths = []
    preds = []
    
    for x, y in dataset.take(steps):
        if filter_models is not None:
            filter_y_pred = filter_models[0].predict(x)
            
            for filter_model in filter_models[1:]:
                filter_y_pred += filter_models[0].predict(x)
                
            filter_y_pred = filter_y_pred / len(filter_models)
            
            filter_y_pred = filter_y_pred > filter_cutoff
            filter_y_pred = filter_y_pred.astype(np.int32)
        else:
            filter_y_pred = np.ones(x.shape[0])
            
        y_pred = model.predict(x)
        y_pred = y_pred * filter_y_pred
        
        truths.extend(y.numpy()[:, 0])
        preds.extend(y_pred[:, 0])
    
    truths = np.array(truths)
    preds = np.array(preds)
    
    preds[preds < 0] = 0
    preds[preds > max_output] = max_output
    
    non0_idx = np.where(truths != 0.0)[0]
    
    mae = np.mean(np.absolute(truths - preds))
    mae_std = np.std(np.absolute(truths - preds))
    mse = np.mean(np.square(truths - preds))
    mse_std = np.std(np.square(truths - preds))
    rmse = np.sqrt(mse)
    
    non0_rmse = np.sqrt(np.mean(np.square(truths[non0_idx] - preds[non0_idx])))
    non0_mae = np.mean(np.absolute(truths[non0_idx] - preds[non0_idx]))
    
    print(f'Model: {label} -- \nLoss: {mse} (+/- {mse_std}) \nMAE: {mae} (+/- {mae_std}) \nRMSE: {rmse} \nFilter0-RMSE: {non0_rmse} \nFilter0-MAE: {non0_mae}')

In [30]:
print('Hands Narrowing')

hand_j_filter_model_paths = ['../trained_models/narrowing/v7/hands_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5', '../trained_models/narrowing/v7/hands_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_test_gamma1.h5']

eval_model('3e-4 (25%, Current Model, 20%):', tf_hands_data[1], '../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 26, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.2)

Hands Narrowing
Model: 3e-4 (25%, Current Model, 20%): -- 
Loss: 0.14468636183578798 (+/- 0.5172341829313837) 
MAE: 0.14110698990631276 (+/- 0.3532353029290359) 
RMSE: 0.38037660526876255 
Filter0-RMSE: 0.7736221920698619 
Filter0-MAE: 0.5806458916886028


In [31]:
eval_model('3e-4 (25%, Current Model, 30%):', tf_hands_data[1], '../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 26, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.3)

Model: 3e-4 (25%, Current Model, 30%): -- 
Loss: 0.14315179275359552 (+/- 0.5227347465248244) 
MAE: 0.134771643510948 (+/- 0.35353698089302266) 
RMSE: 0.37835405740337386 
Filter0-RMSE: 0.7888574316400792 
Filter0-MAE: 0.5940503207120028


In [34]:
filter_model_path = '../trained_models/narrowing/v7/hands_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 40%):', tf_hands_data[1], '../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 26, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.4)

Model: 3e-4 (25%, Current Model, 40%): -- 
Loss: 0.14577924247679055 (+/- 0.5779334347201353) 
MAE: 0.1286447114454439 (+/- 0.3594854387786925) 
RMSE: 0.38181047978911026 
Filter0-RMSE: 0.8462910474458277 
Filter0-MAE: 0.6331717866065698


In [32]:
filter_model_path = '../trained_models/narrowing/v7/hands_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 50%):', tf_hands_data[1], '../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 26, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.5)

Model: 3e-4 (25%, Current Model, 50%): -- 
Loss: 0.15794468839931453 (+/- 0.6189405622479776) 
MAE: 0.1311778744118503 (+/- 0.37514937513489655) 
RMSE: 0.3974225564802714 
Filter0-RMSE: 0.916842812997494 
Filter0-MAE: 0.6945769847471477


In [33]:
eval_model('3e-4 (25%, GAP, 50%):', tf_hands_data[1], '../trained_models/narrowing/v7/hands_narrowing_joint_damage_model_complex_rewritten_gap_64bs_145steps_300epochs_adamW_3e4_1e6_fullwd_min_balanced_mixed_test.h5', 4, 26, filter_model_paths = hand_j_filter_model_paths, filter_cutoff = 0.5)

Model: 3e-4 (25%, GAP, 50%): -- 
Loss: 0.16386721630048146 (+/- 0.6386547006277506) 
MAE: 0.1421366289141588 (+/- 0.3790308628877076) 
RMSE: 0.40480515844104736 
Filter0-RMSE: 0.9204212108942093 
Filter0-MAE: 0.7355700771916996


In [6]:
print('Feet Narrowing')

filter_model_path = '../trained_models/narrowing/v7/feet_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 20%):', tf_feet_j_data[1], '../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 18, filter_model_path = filter_model_path, filter_cutoff = 0.2)

Feet Narrowing
Model: 3e-4 (25%, Current Model, 20%): -- 
Loss: 0.2596522318616103 (+/- 0.9091027755035611) 
MAE: 0.19640637123181173 (+/- 0.4701880147357672) 
RMSE: 0.5095608225340821 
Filter0-RMSE: 1.0563233603265394 
Filter0-MAE: 0.7962802364136362


In [11]:
filter_model_path = '../trained_models/narrowing/v7/feet_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 30%):', tf_feet_j_data[1], '../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 18, filter_model_path = filter_model_path, filter_cutoff = 0.3)

Model: 3e-4 (25%, Current Model, 30%): -- 
Loss: 0.2633512054414539 (+/- 0.930186497756049) 
MAE: 0.1908284234167594 (+/- 0.47637770545831376) 
RMSE: 0.5131775574218478 
Filter0-RMSE: 1.0805629533419228 
Filter0-MAE: 0.8152097343339533


In [7]:
filter_model_path = '../trained_models/narrowing/v7/feet_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 50%):', tf_feet_j_data[1], '../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 4, 18, filter_model_path = filter_model_path)

Model: 3e-4 (25%, Current Model, 50%): -- 
Loss: 0.27968531606351604 (+/- 1.0539183998469839) 
MAE: 0.1855139549006708 (+/- 0.4952473004475925) 
RMSE: 0.5288528302500763 
Filter0-RMSE: 1.174175094909469 
Filter0-MAE: 0.8847736925794388


In [10]:
filter_model_path = '../trained_models/narrowing/v7/feet_narrowing_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Filtered, Balanced GAP, 50%):', tf_feet_j_data[1], '../trained_models/narrowing/v7/feet_narrowing_joint_damage_model_complex_rewritten_gap_64bs_125steps_300epochs_adamW_3e4_1e6_fullwd_min_balanced_mixed_test.h5', 4, 18, filter_model_path = filter_model_path)

Model: 3e-4 (25%, Filtered, Balanced GAP, 50%): -- 
Loss: 0.2795910049318614 (+/- 1.024010699621841) 
MAE: 0.1989587629634318 (+/- 0.48990449637855177) 
RMSE: 0.5287636569695968 
Filter0-RMSE: 1.1561051665193933 
Filter0-MAE: 0.914499932134212


In [12]:
print('Hands Erosion')

filter_model_path = '../trained_models/erosion/v7/hands_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 10%):', tf_hands_e_data[1], '../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 5, 29, filter_model_path = filter_model_path, filter_cutoff = 0.1)

Hands Erosion
Model: 3e-4 (25%, Current Model, 10%): -- 
Loss: 0.37639929973765107 (+/- 1.792490642634114) 
MAE: 0.21398697971920322 (+/- 0.5749859756970638) 
RMSE: 0.6135138953093492 
Filter0-RMSE: 1.9062674106688084 
Filter0-MAE: 1.545595026893329


In [13]:
filter_model_path = '../trained_models/erosion/v7/hands_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 30%):', tf_hands_e_data[1], '../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 5, 29, filter_model_path = filter_model_path, filter_cutoff = 0.3)

Model: 3e-4 (25%, Current Model, 30%): -- 
Loss: 0.3728467900324194 (+/- 1.802687022538263) 
MAE: 0.18591634299935259 (+/- 0.5816200679465646) 
RMSE: 0.6106118161585308 
Filter0-RMSE: 1.9364197504175547 
Filter0-MAE: 1.5953101884054988


In [14]:
filter_model_path = '../trained_models/erosion/v7/hands_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, Current Model, 50%):', tf_hands_e_data[1], '../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_balanced_mixed_test.h5', 5, 29, filter_model_path = filter_model_path, filter_cutoff = 0.5)

Model: 3e-4 (25%, Current Model, 50%): -- 
Loss: 0.39529581294564775 (+/- 2.0104370597419994) 
MAE: 0.1742831717670799 (+/- 0.6040870706979701) 
RMSE: 0.6287255465985518 
Filter0-RMSE: 2.0652230143330623 
Filter0-MAE: 1.732611715416365


In [26]:
filter_model_path = '../trained_models/erosion/v7/hands_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('3e-4 (25%, GAP, 50%):', tf_hands_e_data[1], '../trained_models/erosion/v7/hands_erosion_joint_damage_model_complex_rewritten_gap_64bs_105steps_300epochs_adamW_3e4_1e6_fullwd_min_balanced_mixed_test.h5', 5, 29, filter_model_path = filter_model_path, filter_cutoff = 0.5)

Model: 3e-4 (25%, GAP, 50%): -- 
Loss: 0.3895602033593564 (+/- 1.956488321325006) 
MAE: 0.17864221872175753 (+/- 0.5980360867452098) 
RMSE: 0.6241475813935006 
Filter0-RMSE: 2.0140094401809754 
Filter0-MAE: 1.7070911629007588


In [15]:
print('Feet Erosion')

filter_model_path = '../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('1e-3 (25%, Current Model, 20%):', tf_feet_e_data[1], '../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5', 10, 18, filter_model_path = filter_model_path, filter_cutoff = 0.2)

Feet Erosion
Model: 1e-3 (25%, Current Model, 20%): -- 
Loss: 0.9304040825995575 (+/- 4.78174783061611) 
MAE: 0.37250533308057737 (+/- 0.8897437043475417) 
RMSE: 0.9645745604148792 
Filter0-RMSE: 2.2568814992746167 
Filter0-MAE: 1.649642216764414


In [16]:
filter_model_path = '../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('1e-3 (25%, Current Model, Cutoff 30%):', tf_feet_e_data[1], '../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5', 10, 18, filter_model_path = filter_model_path, filter_cutoff = 0.3)

Model: 1e-3 (25%, Cutoff 30%): -- 
Loss: 0.9582279479840748 (+/- 4.891644846403791) 
MAE: 0.3583757384913042 (+/- 0.910930720771282) 
RMSE: 0.9788911829126232 
Filter0-RMSE: 2.3096839023292923 
Filter0-MAE: 1.6882105513890082


In [19]:
filter_model_path = '../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('1e-3 (25%, Current Model, Cutoff 40%):', tf_feet_e_data[1], '../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5', 10, 18, filter_model_path = filter_model_path, filter_cutoff = 0.4)

Model: 1e-3 (25%, Current Model, Cutoff 40%): -- 
Loss: 0.9373362555598317 (+/- 4.871533582902528) 
MAE: 0.3368311959525777 (+/- 0.9076789085315289) 
RMSE: 0.9681612755940158 
Filter0-RMSE: 2.3456750575197107 
Filter0-MAE: 1.730232214325986


In [17]:
filter_model_path = '../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('1e-3 (25%, Current Model, Cutoff 50%):', tf_feet_e_data[1], '../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_64bs_140steps_mixed_balanced_test_1e3.h5', 10, 18, filter_model_path = filter_model_path, filter_cutoff = 0.5)

Model: 1e-3 (25%, Current Model, Cutoff 50%): -- 
Loss: 0.9917547380062379 (+/- 5.230132023155142) 
MAE: 0.3301995723482428 (+/- 0.9395333844133882) 
RMSE: 0.9958688357440642 
Filter0-RMSE: 2.4448673931705227 
Filter0-MAE: 1.7965282728243326


In [20]:
filter_model_path = '../trained_models/erosion/v7/feet_erosion_joint_damage_type_model_complex_rewritten_64bs_normsteps_75epochs_adamW_3e4_1e6_mixed_test_calpha.h5'

eval_model('1e-3 (25%, balanaced, GAP, 50%):', tf_feet_e_data[1], '../trained_models/erosion/v7/feet_erosion_joint_damage_model_complex_rewritten_gap_64bs_140steps_300epochs_adamW_1e3_1e6_fullwd_balanced_mixed_test.h5', 10, 18, filter_model_path = filter_model_path)

Model: 1e-3 (25%, balanaced, GAP, 50%): -- 
Loss: 0.9385640837129964 (+/- 4.63938974260766) 
MAE: 0.3374411317118857 (+/- 0.9081286067193337) 
RMSE: 0.9687951711858376 
Filter0-RMSE: 2.377565739136836 
Filter0-MAE: 1.7565374467463148
